In [5]:
# Importing Dependencies
import numpy as np
from typing import Tuple

# Defining class for Needleman-Wunsch Algorithm for Global pairwise alignment
class NeedlemanWunsch:
    """ Class for NeedlemanWunsch Alignment

    Parameters:
        sub_matrix_file: str
            Path/filename of substitution matrix
        gap_open: float
            Gap opening penalty
        gap_extend: float
            Gap extension penalty

    Attributes:
        seqA_align: str
            seqA alignment
        seqB_align: str
            seqB alignment
        alignment_score: float
            Score of alignment from algorithm
        gap_open: float
            Gap opening penalty
        gap_extend: float
            Gap extension penalty
    """
    def __init__(self, sub_matrix_file: str, gap_open: float, gap_extend: float):
        # Init alignment and gap matrices
        self._align_matrix = None
        self._gapA_matrix = None
        self._gapB_matrix = None

        # Init matrices for backtrace procedure
        self._back = None
        self._back_A = None
        self._back_B = None

        # Init alignment_score
        self.alignment_score = 0

        # Init empty alignment attributes
        self.seqA_align = ""
        self.seqB_align = ""

        # Init empty sequences
        self._seqA = ""
        self._seqB = ""

        # Setting gap open and gap extension penalties
        self.gap_open = gap_open
        assert gap_open < 0, "Gap opening penalty must be negative."
        self.gap_extend = gap_extend
        assert gap_extend < 0, "Gap extension penalty must be negative."

        # Generating substitution matrix
        self.sub_dict = self._read_sub_matrix(sub_matrix_file) # substitution dictionary

    def _read_sub_matrix(self, sub_matrix_file):
        """
        DO NOT MODIFY THIS METHOD! IT IS ALREADY COMPLETE!

        This function reads in a scoring matrix from any matrix like file.
        Where there is a line of the residues followed by substitution matrix.
        This file also saves the alphabet list attribute.

        Parameters:
            sub_matrix_file: str
                Name (and associated path if not in current working directory)
                of the matrix file that contains the scoring matrix.

        Returns:
            dict_sub: dict
                Substitution matrix dictionary with tuple of the two residues as
                the key and score as value e.g. {('A', 'A'): 4} or {('A', 'D'): -8}
        """
        with open(sub_matrix_file, 'r') as f:
            dict_sub = {}  # Dictionary for storing scores from sub matrix
            residue_list = []  # For storing residue list
            start = False  # trigger for reading in score values
            res_2 = 0  # used for generating substitution matrix
            # reading file line by line
            for line_num, line in enumerate(f):
                # Reading in residue list
                if '#' not in line.strip() and start is False:
                    residue_list = [k for k in line.strip().upper().split(' ') if k != '']
                    start = True
                # Generating substitution scoring dictionary
                elif start is True and res_2 < len(residue_list):
                    line = [k for k in line.strip().split(' ') if k != '']
                    # reading in line by line to create substitution dictionary
                    assert len(residue_list) == len(line), "Score line should be same length as residue list"
                    for res_1 in range(len(line)):
                        dict_sub[(residue_list[res_1], residue_list[res_2])] = float(line[res_1])
                    res_2 += 1
                elif start is True and res_2 == len(residue_list):
                    break
        return dict_sub
    
    def align(self, seqA: str, seqB: str) -> Tuple[float, str, str]:

        # Resetting alignment in case method is called more than once
        self.seqA_align = ""
        self.seqB_align = ""

        # Resetting alignment score in case method is called more than once
        self.alignment_score = 0

        # Initializing sequences for use in backtrace method
        self._seqA = seqA
        self._seqB = seqB

        # Initialize matrix private attributes for use in alignment
        n = len(seqA)
        m = len(seqB)

        self._align_matrix = np.zeros((n + 1, m + 1))
        self._gapA_matrix = np.zeros((n + 1, m + 1))
        self._gapB_matrix = np.zeros((n + 1, m + 1))
        self._back = np.zeros((n + 1, m + 1), dtype=int)
        self._back_A = np.zeros((n + 1, m + 1), dtype=int)
        self._back_B = np.zeros((n + 1, m + 1), dtype=int)

        for i in range(n + 1):
            self._align_matrix[i][0] = i * self.gap_open
            self._gapA_matrix[i][0] = float('-inf')
        for j in range(m + 1):
            self._align_matrix[0][j] = j * self.gap_open
            self._gapB_matrix[0][j] = float('-inf')

        # Compute alignment score and backtrace
        for i in range(1, n + 1):
            for j in range(1, m + 1):
                match = self._align_matrix[i - 1][j - 1] + self.sub_dict[(seqA[i - 1], seqB[j - 1])]
                delete = max(self._gapA_matrix[i - 1][j] + self.gap_extend, self._align_matrix[i-1][j] + self.gap_open)
                insert = max(self._gapB_matrix[i][j - 1] + self.gap_extend, self._align_matrix[i][j-1] + self.gap_open)

#checking matrix generation step by step
                #print("i:", i, "j:", j)
                #print("match:", match, "delete:", delete, "insert:", insert)
                #print("sub_dict:", self.sub_dict)
                #print("seqA[i - 1]:", seqA[i - 1], "seqB[j - 1]:", seqB[j - 1])

                # Update matrices
                self._align_matrix[i][j] = max(match, delete, insert)
                if self._align_matrix[i][j] == match:
                    self._back[i][j] = 0
                    self._back_A[i][j] = i - 1
                    self._back_B[i][j] = j - 1
                elif self._align_matrix[i][j] == delete:
                    self._back[i][j] = 1
                    self._back_A[i][j] = i - 1
                    self._back_B[i][j] = j
                else:
                    self._back[i][j] = 2
                    self._back_A[i][j] = i
                    self._back_B[i][j] = j - 1

 #checking matrix generation step by step                   
                #print("_align_matrix:", self._align_matrix)
                #print("_gapA_matrix:", self._gapA_matrix)
                #print("_gapB_matrix:", self._gapB_matrix)
                #print("_back:", self._back)
                #print("_back_A:", self._back_A)
                #print("_back_B:", self._back_B)

        return self._backtrace()

    def _backtrace(self) -> Tuple[float, str, str]:

        # Start at the bottom-right corner of the backtrace matrix
        i = len(self._seqA)
        j = len(self._seqB)

        # While the current position is not in the top row or left column:
        while i > 0 or j > 0:
            if self._back[i][j] == 0:
                # match - append characters to both aligned sequences and move diagonally
                self.seqA_align = self._seqA[i - 1] + self.seqA_align
                self.seqB_align = self._seqB[j - 1] + self.seqB_align
                i -= 1
                j -= 1
            elif self._back[i][j] == 1:
                # gap in sequence A - append gap to seqB_align and move up
                self.seqA_align = self._seqA[i - 1] + self.seqA_align
                self.seqB_align = "-" + self.seqB_align
                i -= 1
            elif self._back[i][j] == 2:
                # gap in sequence B - append gap to seqA_align and move left
                self.seqA_align = "-" + self.seqA_align
                self.seqB_align = self._seqB[j - 1] + self.seqB_align
                j -= 1

        # Set the alignment score to the value in the bottom-right corner of the alignment matrix
        self.alignment_score = self._align_matrix[-1][-1]

        # Return the alignment score and the aligned sequences
        return (self.alignment_score, self.seqA_align, self.seqB_align)
    
def read_fasta(fasta_file: str) -> Tuple[str, str]:
    """
    DO NOT MODIFY THIS FUNCTION! IT IS ALREADY COMPLETE!

    This function reads in a FASTA file and returns the associated
    string of characters (residues or nucleotides) and the header.
    This function assumes a single protein or nucleotide sequence
    per fasta file and will only read in the first sequence in the
    file if multiple are provided.

    Parameters:
        fasta_file: str
            name (and associated path if not in current working directory)
            of the Fasta file.

    Returns:
        seq: str
            String of characters from FASTA file
        header: str
            Fasta header
    """
    assert fasta_file.endswith(".fa"), "Fasta file must be a fasta file with the suffix .fa"
    with open(fasta_file) as f:
        seq = ""  # initializing sequence
        first_header = True
        for line in f:
            is_header = line.strip().startswith(">")
            # Reading in the first header
            if is_header and first_header:
                header = line.strip()  # reading in fasta header
                first_header = False
            # Reading in the sequence line by line
            elif not is_header:
                seq += line.strip().upper()  # generating full sequence
            # Breaking if more than one header is provided in the fasta file
            elif is_header and not first_header:
                break
    return seq, header

In [6]:
# Define the parameters for the alignment
sub_matrix_file = "../substitution_matrices/BLOSUM62.mat"
gap_open = -10
gap_extend = -1

# Create an instance of the NeedlemanWunsch class
nw = NeedlemanWunsch(sub_matrix_file, gap_open, gap_extend)

# Define the input sequences to be aligned
seqA = "MAVHQLIRRP"
seqB = "MQLIRHP"

# Perform the alignment
alignment_score, seqA_align, seqB_align = nw.align(seqA, seqB)

# Print the results
print("Alignment score: ", alignment_score)
print("Sequence A alignment: ", seqA_align)
print("Sequence B alignment: ", seqB_align)

_back_A: [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
_back_A: [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
_back_A: [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
_back_A: [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
_back_A: [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 